# MINST 手写数字分类
项目来源：[PyTorch手写数字识别](https://blog.csdn.net/weixin_45954454/article/details/114519299)
## 导入必要的包

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torchvision 

train_Data = torchvision.datasets.MNIST('./Data/',train = True,transform = torchvision.transforms.ToTensor(),download = True)
test_Data = torchvision.datasets.MNIST('./Data/',train = False,transform = torchvision.transforms.ToTensor(),download = True)


print(train_Data.classes)
train_data_size = len(train_Data)
test_data_size = len(test_Data)
print("训练数据集长度{}".format(train_data_size))
print("测试数据集长度{}".format(test_data_size))

img, label = train_Data[4]
print(label)

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
训练数据集长度60000
测试数据集长度10000
9


In [ ]:
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
train_dataloader = DataLoader(train_Data,batch_size=64)
test_dataloader = DataLoader(test_Data,batch_size=64)

img,target = train_Data[4]
print(target,img.shape)

to_pil = transforms.ToPILImage()

plt_img = to_pil(img)
plt.imshow(plt_img,cmap='gray')
plt.show()


## 搭建神经网络
![模型框架](images/MINST模型框架.png)

In [ ]:
from MINST_Model import *

net = Net()

loss_fn = nn.CrossEntropyLoss()

learning_rate = 0.01
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate)

## 训练及保存模型

In [ ]:
from tqdm import tqdm
# 设置训练网络的参数
total_train_step = 0    # 训练的次数
total_test_step = 0     # 测试的次数
epoch = 100              # 训练的轮数

history_loss = []
history_accuracy = []
history_epoch = []
for i in range(epoch):
    print("----------------------第{}轮训练开始----------------------".format(i+1))
    # 开启训练
    net.train(True)
    for data in tqdm(train_dataloader):
        imgs,targets = data
        outputs = net(imgs)

        loss= loss_fn(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step = total_train_step+1
        # if total_train_step %100==0:
            # print("训练次数：{},loss:{}".format(total_train_step, loss.item()))
    net.eval()
    total_test_loss= 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            output = net(imgs)
            loss = loss_fn(output,targets)
            total_test_loss = total_test_loss+loss.item()
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
    
    print("整体测试集的loss:{}".format(total_test_loss))
    print("整体测试集的正确率：{}".format(total_accuracy/test_data_size))
    total_test_step=total_test_step+1
    
    torch.save(net,"models/MINST_{}.pth".format(i))
	# torch.save(tudui.state_dict(),"models/tudui_{}.pth".format(i))
    # print("模型已保存")
    history_loss.append(total_test_loss)
    history_accuracy.append(total_accuracy/test_data_size)
    history_epoch.append(i)
print("训练结束")

## 打印可视化log图

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_epoch,history_accuracy, label='accuracy')
plt.show()